In [6]:
from scidx_streaming import StreamingClient
from ndp_ep import APIClient
import os, datetime
from dotenv import load_dotenv
# ---- Configuration ----
load_dotenv(override=True)
TOKEN = os.getenv("TOKEN")
API_URL = os.getenv("API_URL")
SERVER = os.getenv("SERVER")
SYNOPTIC_TOKEN = os.getenv("SYNOPTIC_TOKEN")
# initializing ndp_ep APIClient
client = APIClient(base_url=API_URL, token=TOKEN)
streaming = StreamingClient(client)
print(f"Streaming Client initialized. User ID: {streaming.user_id}")
date_time_now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
org_name = "test"

Streaming Client initialized. User ID: 987104e7-e6d3-47f2-82a0-0d3f620aea70


In [ ]:
org_data = {
        "name": org_name,
        "title": org_name,
        "description": "Sumaiya test organization for testing purposes",
    }
client.register_organization(org_data,server=SERVER)

In [7]:
dataset_metadata = {
    "name": "synoptic_push_utah_aq",
    "title": "Synoptic Push Utah Air Quality",
    "notes": "Real-time air quality observations from Synoptic Push (WebSocket).",
    "owner_org": org_name,
}
synoptic_ws_method = {
    # "type": "csv",
    "type": "websocket",
    "name": "synoptic-push-wss",
    "description": "Synoptic Push WebSocket stream for selected stations and vars.",
    "config": {
        "url": f"wss://push.synopticdata.com/feed/{SYNOPTIC_TOKEN}/?stid=QNR&vars=PM_25_concentration,ozone_concentration,CO_concentration,NO2_concentration,air_temp",
        "token_env": "SYNOPTIC_TOKEN",
    }
}
client.delete_resource_by_name(dataset_metadata['name'],server=SERVER)
ds = streaming.register_data_source(
        dataset_metadata=dataset_metadata,
        methods=[synoptic_ws_method],
        server=SERVER,   # or whatever you use
    )
print(f"Dataset registered with ID: {ds['id']}")
results = streaming.search_consumption_methods(terms=["synoptic"]) 
print(f"Search returned {len(results)} results. {results[0]}")

Dataset registered with ID: e636c4c7-ef55-46d4-8ff1-2f0b44159c75
Search returned 6 results. {'name': 'synoptic_push_qcv_aq', 'description': 'Real-time air quality observations from Synoptic Push for station QCV (Copperview).', 'resources': [{'id': '6f5e07c1-9ffe-4357-89b7-b38f3a630684', 'type': 'websocket', 'name': 'synoptic-push-wss-qcv', 'description': 'Synoptic Push WebSocket stream for QCV.', 'config': {'url': 'wss://push.synopticdata.com/feed/e3636084e69244f5bd4d4d2721cec435/?stid=QCV&vars=PM_25_concentration,ozone_concentration,CO_concentration,NO2_concentration', 'token_env': 'SYNOPTIC_TOKEN', 'station_id': 'QCV', 'station_name': 'Copperview', 'latitude': 40.59806, 'longitude': -111.89417, 'elevation_m': 1343.9}}]}


In [8]:
stream = await streaming.create_kafka_stream(
    consumption_method_ids = [results[0]["resources"][0]["id"]],
    filter_semantics=[],
    server = SERVER,
)

topic = stream.data_stream_id
print(f"Stream created: {topic}")

Stream created: data_stream_987104e7-e6d3-47f2-82a0-0d3f620aea70_4


[websocket] CONNECTED url=wss://push.synopticdata.com/feed/e3636084e69244f5bd4d4d2721cec435/?stid=QCV&vars=PM_25_concentration,ozone_concentration,CO_concentration,NO2_concentration
[websocket] batch -> records=1 msg_types={'auth': 1}


In [ ]:
import asyncio
import json
from pprint import pprint
from datetime import datetime, timedelta, timezone

from scidx_streaming import StreamingClient
from ndp_ep import APIClient
import os
from dotenv import load_dotenv

# ----------------------------
# Configuration
# ----------------------------
load_dotenv(override=True)

TOKEN = os.getenv("TOKEN")
API_URL = os.getenv("API_URL")
SERVER = os.getenv("SERVER")

client = APIClient(base_url=API_URL, token=TOKEN)
streaming = StreamingClient(client)

print(f"Streaming Client initialized. User ID: {streaming.user_id}")

# ----------------------------
# Stream (already created)
# ----------------------------
topic = "data_stream_987104e7-e6d3-47f2-82a0-0d3f620aea70_1"
print(f"Consuming from topic: {topic}")

consumer = streaming.consume_kafka_messages(topic)

# ----------------------------
# Helpers
# ----------------------------
def pretty_print_msg(msg: dict):
    """Nice readable printing for websocket messages"""
    print("\n" + "=" * 80)
    pprint(msg, width=120, sort_dicts=False)
    print("=" * 80 + "\n")


def normalize_payload_cell(cell):
    """
    payload column may contain:
    - dict
    - list of dicts / None
    """
    if isinstance(cell, dict):
        return [cell]
    if isinstance(cell, list):
        return [x for x in cell if isinstance(x, dict)]
    return []


# ----------------------------
# Main loop
# ----------------------------
async def main():
    print("Waiting for data...\n")

    end_time = datetime.now(timezone.utc) + timedelta(hours=1)

    # Track what we have already printed
    seen_messages = set()
    last_df_signature = None

    while datetime.now(timezone.utc) < end_time:
        df = consumer.dataframe

        if df is not None and not df.empty:
            # ---------- Print dataframe only when it updates ----------
            signature = (len(df), df["timestamp"].iloc[-1] if "timestamp" in df.columns else None)
            if signature != last_df_signature:
                last_df_signature = signature
                print("\n📊 Updated DataFrame (latest rows):")
                print(df.tail(5).to_string(index=False))

            # ---------- Pretty-print raw websocket messages ----------
            if "payload" in df.columns:
                for cell in df["payload"].dropna():
                    for msg in normalize_payload_cell(cell):
                        # Create a stable hash so we don't print duplicates
                        msg_key = json.dumps(msg, sort_keys=True, default=str)

                        if msg_key not in seen_messages:
                            seen_messages.add(msg_key)
                            pretty_print_msg(msg)

        await asyncio.sleep(2)

    consumer.stop()
    print("⏹️ No new data after 1 hour. Stopping consumer.")


# ----------------------------
# Run
# ----------------------------
await main()


Streaming Client initialized. User ID: 987104e7-e6d3-47f2-82a0-0d3f620aea70
Consuming from topic: data_stream_987104e7-e6d3-47f2-82a0-0d3f620aea70_1
Waiting for data...



In [ ]:
consumer.stop()

In [ ]:
import requests
import time
KEYCLOAK_URL = "https://idp-test.nationaldataplatform.org/realms/NDP"
CLIENT_ID = "saleem_test_delete"  # Should be public (no secret)
SCOPE = "openid profile email"
def get_user_token_device_flow():
    device_endpoint = f"{KEYCLOAK_URL}/protocol/openid-connect/auth/device"
    token_endpoint = f"{KEYCLOAK_URL}/protocol/openid-connect/token"
    # Step 1: Initiate Device Flow
    response = requests.post(device_endpoint, data={
        "client_id": CLIENT_ID,
        "scope": SCOPE
    })
    response.raise_for_status()
    data = response.json()
    print(f"\nPlease open this URL in your browser: {data['verification_uri']}")
    print(f"Enter this code when prompted: {data['user_code']}\n")
    # Step 2: Poll until approved
    while True:
        time.sleep(data["interval"])
        poll = requests.post(token_endpoint, data={
            "grant_type": "urn:ietf:params:oauth:grant-type:device_code",
            "device_code": data["device_code"],
            "client_id": CLIENT_ID
        })
        if poll.status_code == 200:
            print("Token received!")
            return poll.json()
        elif poll.status_code == 400 and poll.json().get("error") == "authorization_pending":
            continue
        else:
            raise Exception(f"Failed: {poll.status_code} {poll.text}")
# === Get the token ===
token_response = get_user_token_device_flow()
print("Access Token:", token_response["access_token"])
TOKEN = token_response["access_token"]
print(TOKEN)

In [ ]:
from websocket import create_connection
import json

my_token = "e3636084e69244f5bd4d4d2721cec435"
stream_server = "wss://push.synopticdata.com/feed/"
# query_arguments = "units=english&stid=WBB&vars=air_temp,relative_humidity"
query_arguments = "stid=QNR&vars=PM_25_concentration,ozone_concentration,CO_concentration,NO2_concentration,air_temp"

# then you simply create your connection
ws = create_connection(stream_server+my_token+"/?"+query_arguments)


while True:
	data = ws.recv()
	json_data = json.loads(data)
	print(json_data)

